In [1]:
import os
import sys

SPARK_HOME = "/usr/lib/spark3"
PYSPARK_PYTHON = "/opt/conda/envs/dsenv/bin/python"
os.environ["PYSPARK_PYTHON"]= PYSPARK_PYTHON
os.environ["PYSPARK_DRIVER_PYTHON"]= PYSPARK_PYTHON
os.environ["SPARK_HOME"] = SPARK_HOME

PYSPARK_HOME = os.path.join(SPARK_HOME, "python/lib")
sys.path.insert(0, os.path.join(PYSPARK_HOME, "py4j-0.10.9.3-src.zip"))
sys.path.insert(0, os.path.join(PYSPARK_HOME, "pyspark.zip"))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()

spark = SparkSession.builder.config(conf=conf).appName("Spark SQL").getOrCreate()

Picked up _JAVA_OPTIONS: 
Picked up _JAVA_OPTIONS: 
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark3/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 12:56:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/05 12:56:32 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/05 12:56:32 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
from pyspark.sql.types import *
schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("follower_id", IntegerType()),
])

In [32]:
df = spark.read\
          .schema(schema)\
          .format("csv")\
          .option("sep", "\t")\
          .load("/datasets/twitter/twitter_sample_small.tsv")

In [5]:
df.take(5)

[Row(user_id=12, follower_id=2241),
 Row(user_id=12, follower_id=13349),
 Row(user_id=12, follower_id=41873),
 Row(user_id=12, follower_id=82473),
 Row(user_id=12, follower_id=414853)]

In [8]:
children = df.filter(df.follower_id == 12).select(df.user_id).rdd.flatMap(lambda x: x).collect()
for el in children:
    if parents.get(el, -1) == -1:
        parents[el] = cur_el

In [14]:
mvv_count_df.select("mvv").rdd.flatMap(lambda x: x).collect()

AttributeError: 'DataFrame' object has no attribute 'values'

In [33]:
def bf_search(df, start, stop):
    queue = []
    parents = {}
    cur_el = start
    success = 0
    queue.append(cur_el)
    while queue:
        cur_el = queue.pop(0)
        children = df.filter(df.follower_id == cur_el).select(df.user_id).rdd.flatMap(lambda x: x).collect()
        if stop in children:
            success = 1
            break
        for el in children:
            if parents.get(el, -1) == -1:
                parents[el] = cur_el
                queue.append(el)
    if (success == 1):
        res = []
        while cur_el != start:
            res.append(cur_el)
            cur_el = parents[cur_el]
    return res

In [34]:
start = int(12)
stop = int(34)

In [35]:
res = bf_search(df, start, stop)

In [36]:
res = [stop] + res + [start]

In [37]:
print(res[::-1])

[12, 422, 53, 52, 107, 20, 23, 274, 34]


In [ ]:
df_rep = spark.createDataFrame(data=res[::-1])
df_rep.write.csv(sys.argv[4], sep=',')